
# 🚚 지연 배송(Late Shipments) 데이터 분석 실습

**가설검정(Hypothesis Testing)** 과 **신뢰구간(Confidence Interval)** 개념을 
실제 데이터(`late_shipments`)에 적용

---
## 💡 분석 목표

의료물품 배송 데이터에서 **지연 배송 비율(late proportion)** 이  
가설로 주어진 **6% (0.06)** 과 유의하게 다른지 판단합니다.

- **귀무가설 (Null Hypothesis) (H₀)**: 지연 배송 비율 = 0.06  
- **대립가설 (Alternative Hypothesis) (H₁)**: 지연 배송 비율 > 0.06


In [1]:

# 라이브러리 불러오기
import pandas as pd
import numpy as np
from scipy.stats import norm

# 데이터 불러오기
late_shipments = pd.read_feather("late_shipments.feather")

# 데이터 확인
print(late_shipments.head())
print(late_shipments['late'].value_counts())


        id       country managed_by  fulfill_via vendor_inco_term  \
0  36203.0       Nigeria   PMO - US  Direct Drop              EXW   
1  30998.0      Botswana   PMO - US  Direct Drop              EXW   
2  69871.0       Vietnam   PMO - US  Direct Drop              EXW   
3  17648.0  South Africa   PMO - US  Direct Drop              DDP   
4   5647.0        Uganda   PMO - US  Direct Drop              EXW   

  shipment_mode  late_delivery late product_group    sub_classification  ...  \
0           Air            1.0  Yes          HRDT              HIV test  ...   
1           Air            0.0   No          HRDT              HIV test  ...   
2           Air            0.0   No           ARV                 Adult  ...   
3         Ocean            0.0   No           ARV                 Adult  ...   
4           Air            0.0   No          HRDT  HIV test - Ancillary  ...   

  line_item_quantity line_item_value pack_price unit_price  \
0             2996.0       266644.00      


## 1️⃣ 표본 비율 계산 (Sample proportion)

- 데이터프레임에서 `"late"` 컬럼의 `"Yes"` 비율을 구합니다.  
- `mean()`을 사용하면 간단히 비율 계산이 가능합니다.

예:  
```python
prop = (df['late'] == "Yes").mean()
```


In [2]:

# 지연 배송 비율 계산
late_prop_samp = (late_shipments['late'] == "Yes").mean()

print("지연 배송 비율 (표본 비율):", late_prop_samp)


지연 배송 비율 (표본 비율): 0.061



## 2️⃣ z-score 계산

- **z-score**는 표본 통계량이 가설 하의 모수로부터 얼마나 떨어져 있는지를 표준화한 값입니다.  
- 계산식:
\[ z = \frac{\hat{p} - p_0}{SE} \]

여기서  
- \( \hat{p} \): 표본 비율  
- \( p_0 \): 가설 비율 (0.06)  
- \( SE \): 표준오차 (bootstrap 분포로부터 계산)

---
### 📘 예제 코드


In [6]:

# 가설 비율 설정
late_prop_hyp = 0.06

# late_shipments 데이터에서 'Yes'/'No' 비율 구하기
late_col = (late_shipments['late'] == "Yes").astype(int).values

# 부트스트랩 반복 (예: 1000번 재추출)
B = 1000
n = len(late_col)
boot_props = []

for _ in range(B):
    sample = np.random.choice(late_col, size=n, replace=True)  # 복원추출
    boot_props.append(sample.mean())

# 결과: bootstrap 분포 리스트
late_shipments_boot_distn = np.array(boot_props)

# bootstrap 분포의 표준오차 계산
std_error = np.std(late_shipments_boot_distn, ddof=1)

# z-score 계산
z_score = (late_prop_samp - late_prop_hyp) / std_error

print(f"z-score: {z_score:.3f}")


z-score: 0.128



## 3️⃣ p-value 계산

- **p-value**는 “귀무가설이 참일 때, 지금처럼 극단적인 결과가 나올 확률”입니다.  
- 본 예제에서는 **단측 검정(one-tailed)** 을 수행합니다.  
  (대립가설: 실제 비율이 0.06보다 크다)

---
### 📘 예제 코드


In [7]:

# p-value 계산 (단측)
p_value = 1 - norm.cdf(z_score, loc=0, scale=1)

print(f"p-value: {p_value:.6f}")

# 해석 기준
alpha = 0.05
if p_value < alpha:
    print("귀무가설 기각 → 지연 배송 비율이 6%보다 높다고 볼 수 있음.")
else:
    print("귀무가설 채택 → 지연 배송 비율이 6%보다 높다고 보기 어려움.")


p-value: 0.449066
귀무가설 채택 → 지연 배송 비율이 6%보다 높다고 보기 어려움.



## 4️⃣ 95% 신뢰구간 계산

- **신뢰구간(Confidence Interval)** 은 모수(진짜 비율)가 들어 있을 것으로 기대되는 구간입니다.  
- Bootstrap 분포를 활용한 **quantile(분위수)** 방법을 사용합니다.


In [8]:

# 95% 신뢰구간 (bootstrap quantile method)
lower = np.quantile(late_shipments_boot_distn, 0.025)
upper = np.quantile(late_shipments_boot_distn, 0.975)

print(f"95% 신뢰구간: ({lower:.4f}, {upper:.4f})")

if late_prop_hyp < lower or late_prop_hyp > upper:
    print("가설 비율 0.06은 신뢰구간 밖에 있음 → 유의한 차이 존재 가능")
else:
    print("가설 비율 0.06은 신뢰구간 안에 있음 → 유의한 차이 없음")


95% 신뢰구간: (0.0460, 0.0770)
가설 비율 0.06은 신뢰구간 안에 있음 → 유의한 차이 없음



---
## ✅ 요약 정리

| 단계 | 내용 | 결과 해석 |
|------|------|------------|
| 1 | 표본 비율 계산 | 전체 배송 중 “지연” 비율 |
| 2 | z-score 계산 | 표본 비율이 가설 비율과 얼마나 다른지 |
| 3 | p-value 계산 | 귀무가설이 참일 때 이런 결과가 나올 확률 |
| 4 | 신뢰구간 계산 | 모수(진짜 비율)의 합리적 범위 |
| **결론** | p-value < 0.05 또는 신뢰구간이 0.06을 포함하지 않으면 → 유의한 차이 |

---
**💡 해석 예시:**  
“지연 배송 비율의 표본 추정치는 0.078이며, 이는 가설 비율 0.06보다 유의하게 높았다 (z=2.1, p=0.018).  
따라서 실제 지연 배송 비율은 6%보다 높다고 결론 내릴 수 있다.”
